<a href="https://colab.research.google.com/github/leeesoraaa/TIL/blob/main/data%20analysis/colab/05_Pandas_groupby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# Groupby: 같은 값을 하나로 묶어 통계 또는 집계 결과를 얻기 위한 기능

In [4]:
df = pd.DataFrame({
    'city':['부산','부산','부산','부산','서울','서울', '서울'],
    'fruits':['apple','orange','banana','banana','apple','apple','banana'],
    'price':[100,200,250,300,150,200,400],
    'quantity':[1,2,3,4,5,6,7]
})
df

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
4,서울,apple,150,5
5,서울,apple,200,6
6,서울,banana,400,7


## get_group(): 그룹 안의 데이터를 확인

In [5]:
df.groupby('city').mean()

<ipython-input-5-1de92c00fe84>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('city').mean()


,price,quantity
city,,
부산,212.5,2.5
서울,250.0,6.0


In [6]:
df.groupby('city').get_group('부산')

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4


## size(): 각 그룹의 사이즈 확인

In [8]:
df.groupby('city').size()

city
부산    4
서울    3
dtype: int64

In [9]:
df.groupby('city').size()['부산']

4

## agg(): Aggregation

In [10]:
df.groupby('city').agg(np.mean)

<ipython-input-10-d3e542c46df5>:1: FutureWarning: The operation <function mean at 0x7edcb9f143a0> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  df.groupby('city').agg(np.mean)


,price,quantity
city,,
부산,212.5,2.5
서울,250.0,6.0


In [11]:
# 가격 평균과 수량은 합계를 동시에 계산
def my_mean(s):
  return np.mean(s)

df.groupby('city').agg({'price':my_mean, 'quantity':sum})

,price,quantity
city,,
부산,212.5,10
서울,250.0,18


### 대륙과 국가별 음주 소비량을 분석한 데이터셋
- 각각의 피처들 간의 상관관계 파악
- 아프리카와 유럽 간의 맥주 소비량 차이 검정

In [29]:
drinks = pd.read_csv('drinks.csv')
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     170 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


In [13]:
drinks.describe()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
count,193.000000,193.000000,193.000000,193.000000
mean,106.160622,80.994819,49.450777,4.717098
std,101.143103,88.284312,79.697598,3.773298
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,1.000000,1.300000
50%,76.000000,56.000000,8.000000,4.200000
75%,188.000000,128.000000,59.000000,7.200000
max,376.000000,438.000000,370.000000,14.400000


In [14]:
# 각 피처들 간의 상관관계 파악 - pearson 상관계수
corr = drinks[['beer_servings', 'wine_servings']].corr(method = 'pearson')
print(corr)

               beer_servings  wine_servings
beer_servings       1.000000       0.527172
wine_servings       0.527172       1.000000


### 여러 피처들 간의 상관계수 분석

In [16]:
cols = ['beer_servings', 'spirit_servings', 'wine_servings', 'total_litres_of_pure_alcohol']
corr = drinks[cols].corr(method='pearson')
print(corr)

                              beer_servings  spirit_servings  wine_servings  \
beer_servings                      1.000000         0.458819       0.527172   
spirit_servings                    0.458819         1.000000       0.194797   
wine_servings                      0.527172         0.194797       1.000000   
total_litres_of_pure_alcohol       0.835839         0.654968       0.667598   

                              total_litres_of_pure_alcohol  
beer_servings                                     0.835839  
spirit_servings                                   0.654968  
wine_servings                                     0.667598  
total_litres_of_pure_alcohol                      1.000000  


In [17]:
# heatmap으로 시각화
import seaborn as sns


In [ ]:
# 산점도 그래프로 시각화: sns.pairplot

### 결측값 처리
- 대륙(continent)의 결측값을 대체값(OT)로 채우는 작업

In [18]:
# 결측여부 확인: isnull().sum()
# 결측값 대체: fillna(대체값)
drinks.isnull().sum()

country                          0
beer_servings                    0
spirit_servings                  0
wine_servings                    0
total_litres_of_pure_alcohol     0
continent                       23
dtype: int64

In [19]:
drinks['continent'] = drinks['continent'].fillna('OT')
drinks.head(10)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
5,Antigua & Barbuda,102,128,45,4.9,OT
6,Argentina,193,25,221,8.3,SA
7,Armenia,21,179,11,3.8,EU
8,Australia,261,72,212,10.4,OC
9,Austria,279,75,191,9.7,EU


In [20]:
# 대륙별로 맥주 소비량의 최대, 최소, 평군을 추출
res = drinks.groupby('continent')['beer_servings'].agg(['max', 'min', 'mean'])
res

,max,min,mean
continent,,,
AF,376,0,61.471698
AS,247,0,37.045455
EU,361,0,193.777778
OC,306,0,89.687500
OT,285,1,145.434783
SA,333,93,175.083333


In [22]:
# idxmax():Series 객체에서 값이 가장 큰 index를 반환
# 평균 맥주 소비량이 가장 큰 대륙은?
res = drinks.groupby('continent')['beer_servings'].mean().idxmax()
res

'EU'

In [25]:
# t-test: 차이검정
# 아프리카와 유럽 간의 맥주 소비량 차이 검정
from scipy import stats # import scipy.stats as stats
africa = drinks[drinks['continent'] == 'AF']
europe = drinks[drinks['continent'] == 'EU']

In [27]:
# 정규분포 - 정규성
# shapiro test - 정규성 검증, p-value > 0.05(임계값)  숫자 데이터만 가능
# 귀무가설/대립가설
# p-value < 0.05 대립가설 채택, 정규분포를 따르지 않는다.
sha_a, sha_pvalue = stats.shapiro(africa['beer_servings'])  # 통계량값과 p-value 두 개가 반환
print(sha_a, sha_pvalue) ## 정규분포를 따르지 않는다.

0.6982115507125854 3.84315423929138e-09


In [28]:
# 등분산: 분산이 같다
## levene test
e, p = stats.levene(africa['beer_servings'], europe['beer_servings'])
print(e, p)
if p > 0.05:
  print('등분산이다.')
else:
  print('등분산이 아니다.')

6.015144059118397 0.015989187968374834
등분산이 아니다.


# 카이제곱 검정(Chi square test)
- 두 데이터가 통계적으로 같은지 다른지 검정하는 방법
- 카이제곱 검정에는 두 가지 형태가 있으며, 카이제곱 통계량과 분포를 사용하지만 목적에 따라 달라짐
- 적합도 검정: 관찰된 값이 기댓값과 같은지 다른지 검정
- 동질성 검정: 두 집단의 분포가 동일한지 검정
- 독립성 검정: 두 개 이상의 변수가 서로 독립인지 검정

In [30]:
data1 = [4, 6, 17, 16, 8, 9]
data2 = [10, 10, 10, 10, 10, 10]

# 집단의 대표값
print(np.mean(data1))
print(np.mean(data2))

10.0
10.0


In [31]:
# 카이제곱 검정: 두 데이터가 통계적으로 같은지를 검정
chis = stats.chisquare(data1, data2)
chis  # 통계량, p-value

Power_divergenceResult(statistic=14.200000000000001, pvalue=0.014387678176921308)

- 귀무가설: p-value > 0.05 (통계적으로 같다.)
- 대립가설: p-value < 0.05 (통계적으로 같지 않다.)

# T-검정: 차이 검정
1. 단일표본 t검정(1개의 그룹): 연속형 변수를 대상으로
  - ttest_1samp()

2. 독립표본 t검정(서로 다른 2개의 그룹)
  - ttest_ind()

3. 대응표본 t검정(같은 그룹)
  - ttest_rel()

In [34]:
from numpy.polynomial.hermite_e import hermeweight
# 단일표본 t검정: 연속형 변수인 1개의 그룹
# 일반적으로 청소년의 신장은 175이다.
np.random.seed(100)
# 평균 0, 표준편차 5인 20개의 난수 발생
height = [175+np.random.normal(0, 5) for a in range(20)]
print(height)

[166.25117263472652, 176.7134020166375, 180.7651790128182, 173.73781981739305, 179.90660393475616, 177.5710942069719, 176.105898346107, 169.64978334715855, 174.05252084588412, 176.2750072213669, 172.70986507248688, 177.17581744061445, 172.0820247483867, 179.08423535842888, 178.3636040285483, 174.47794428304687, 172.34359811574046, 180.14866342566674, 172.8093218864779, 169.4084087687228]


In [35]:
result = stats.ttest_1samp(height, 175)
print(result)  # 통계량, p-value

TtestResult(statistic=-0.020929720802605085, pvalue=0.9835199566394546, df=19)


- p-value < 0.05, 대립가설 채택
- p-value > 0.05, 귀무가설 채택

In [36]:
# 독립표본 t검정: 서로 다른 2개의 집단
group1 = [170+np.random.normal(0, 5) for a in range(20)]
group2 = [175+np.random.normal(0, 5) for a in range(20)]
print(group1)
print(group2)

[178.0949083033763, 177.70802587256702, 168.74060430393394, 165.7878213087435, 170.9225934528197, 174.68541100554475, 173.65500171917404, 176.80778062572665, 168.3688097039885, 170.27838007427388, 171.11199804277652, 162.7839150238733, 166.21823847202782, 174.08227005509642, 173.7522238076709, 167.72026536266, 175.94811134014563, 161.54691586808198, 163.21800475569344, 163.83782743042536]
[172.27780419163767, 171.65914131593286, 175.0365728161445, 171.9353063226092, 181.49874037377654, 166.3345218817336, 170.08344950435185, 176.7875387658368, 166.93210748588913, 182.35356933306065, 169.0599120134114, 172.25126903232254, 170.29976919227616, 170.86033817817065, 175.5443173391684, 177.5390479524616, 170.6888632674476, 181.2473487136349, 174.60194377041302, 170.55134259367483]


In [37]:
# 등분산 가정(분산이 같음)
result1 = stats.ttest_ind(group1, group2)
print(result1)

# 등분산 가정을 하지 않을 경우(분산이 다름)
result2 = stats.ttest_ind(group1, group2, equal_var=False)
print(result2)

TtestResult(statistic=-1.9924662346475654, pvalue=0.053539257776348465, df=38.0)
TtestResult(statistic=-1.9924662346475654, pvalue=0.05365506048684418, df=37.40586696691316)


In [40]:
# 대응표본 t검정: 같은 그룹을 대상으로 , 반드시 샘플 데이터의 수가 동일해야 한다.
before = [60+np.random.normal(0, 5) for a in range(20)]
print(before)

after = [60 + w * np.random.normal(0.99, 0.02) for w in range(20)]
print(after)

[55.59100805258491, 60.0931947474403, 61.18922310961811, 60.067742743143064, 51.82235300309589, 54.77895061145342, 63.06519440843773, 63.68102606661912, 65.13460719699896, 52.839046944705366, 50.794058499066416, 61.83046613083652, 58.341114324735955, 56.55391010955123, 70.17303780752466, 57.24642794042703, 63.75226665163421, 53.46503830459591, 62.90286667897137, 54.47738453668853]
[60.0, 61.00373780132277, 61.91733249881686, 63.024298447288004, 64.02230579194584, 64.99282328705968, 65.95306463878775, 66.93395970887524, 67.82738786803344, 68.6940987841451, 69.55880959885236, 70.97121607056354, 72.33037762247092, 72.77200512895607, 74.37294210291115, 74.85090523020936, 75.81567249096813, 76.83134558195579, 77.75339492007704, 77.86488241661534]


In [41]:
res = stats.ttest_rel(before, after)
print(res)
# p-value < 0.05: 대립가설 채택
# p-value > 0.05: 귀무가설 채택

TtestResult(statistic=-6.32940100060346, pvalue=4.493316856757048e-06, df=19)


In [43]:
# 아프리카와 유럽 간의 맥주 소비량 차이를 검정
a = stats.ttest_ind(africa['beer_servings'], europe['beer_servings'])  # 등분산
b = stats.ttest_ind(africa['beer_servings'], europe['beer_servings'], equal_var=False) # 등분산x
print(a)
print(b)

TtestResult(statistic=-7.267986335644365, pvalue=9.719556422442453e-11, df=96.0)
TtestResult(statistic=-7.143520192189803, pvalue=2.9837787864303205e-10, df=84.40013075489844)
